In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing libraries required

In [2]:
import matplotlib.pyplot as plt                  #Data Visualization
from tqdm import tqdm_notebook
import seaborn as sns
import pickle
import joblib
import pywt           #PyWavelets is a free Open Source library for wavelet transforms in Python
import warnings
warnings.filterwarnings("ignore")                  # Donot display warning messages.
from sklearn.preprocessing import StandardScaler   #Data Scaling 
from sklearn.model_selection import GridSearchCV   #Hyperparameter optimization
from sklearn.svm import NuSVR, SVR                 #Support vector Machine Model
from sklearn.kernel_ridge import KernelRidge       #kernel ridge model
from catboost import CatBoostRegressor, Pool       #Catboost Model
import xgboost as xgb                              #XgBoost Model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from prettytable import PrettyTable
from sklearn.model_selection import KFold

In [3]:
from flask import Flask,jsonify,request,render_template
import joblib
app=Flask(__name__)

In [ ]:
@app.route("/")
def hello_world():
  return 'Hello World!'

In [5]:
@app.route('/index')
def index():
  return Flask.render_template('index.html')

In [ ]:
@app.route('/predict',methods=['POST'])
def predict():
  clf=joblib.load('model.pkl')
  pred=clf.predict()
  

In [4]:
train = pd.read_csv('/kaggle/input/LANL-Earthquake-Prediction/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32},error_bad_lines=False)
train.head()

In [5]:
#visualizing first 1000 datapoints
train_ad_df = train['acoustic_data'].values[::1000]
train_ttf_df = train['time_to_failure'].values[::1000]

#function for plotting based on both features
def plot_ad_ttf_data(train_ad_df, train_ttf_df, title="Acoustic data and time to failure: first 1000 sampled data"):
    fig, ax1 = plt.subplots(figsize=(12, 8))
    plt.title(title)
    plt.plot(train_ad_df, color='r')
    ax1.set_ylabel('acoustic data', color='r')
    plt.legend(['acoustic data'], loc=(0.01, 0.95))
    ax2 = ax1.twinx()
    plt.plot(train_ttf_df, color='g')
    ax2.set_ylabel('time to failure', color='b')
    plt.legend(['time to failure'], loc=(0.01, 0.9))
    plt.grid(True)

plot_ad_ttf_data(train_ad_df, train_ttf_df)
del train_ad_df
del train_ttf_df

In the above figure by plotting first 1000 datapoints we can clearly see that at the time of failure there is spike in acoustic data or seismic signal.

In [6]:
train.shape[0]
train_ad_sample_df = train['acoustic_data'].values[:6291454]
train_ttf_sample_df = train['time_to_failure'].values[:6291454]
plot_ad_ttf_data(train_ad_sample_df, train_ttf_sample_df, title="Acoustic data and time to failure")

In [7]:
def mad(x, axis=None):
    """
    Mean Absolute Deviation
    """
    
    return np.mean(np.absolute(x - np.mean(x, axis)), axis)

#https://www.kaggle.com/ilu000/1-private-lb-kernel-lanl-lgbm/
#https://www.kaggle.com/tarunpaparaju/lanl-earthquake-prediction-signal-denoising/notebook
Since data generated by seismogram is a mixture of artificial and actuals signals. Hence we need to denoise the acoustic data to get actual data. This actual underlying signal would be a better predictor of earthquake timing than the original raw signal, because it represents the actual seismic activity.
Since acoustic data can be prone to noise, hence denoising the data can be a good move.  

In [8]:

def denoise_signal(x, wavelet='db4', level=1):
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec(x, wavelet, mode="per")
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf

    sigma = (1/0.6745) * mad(coeff[-level])

    # Calculate the univeral threshold
    u_thresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=u_thresh, mode='hard') for i in coeff[1:])
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec(coeff, wavelet, mode='per')

In [9]:
train_ad_df = train['acoustic_data'].values[::50000]
train_ttf_df = train['time_to_failure'].values[::50000]

#function for plotting based on both features
def plot_ad_ttf_data(x,y, title="Acoustic data and time to failure after denoising: first 50000 sampled data"):
    fig, ax1 = plt.subplots(figsize=(12, 8))
    plt.title(title)
    plt.plot(denoise_signal(x), color='r')
    ax1.set_ylabel('acoustic data', color='r')
    plt.legend(['acoustic data'], loc=(0.01, 0.95))
    ax2 = ax1.twinx()
    plt.plot(y, color='g')
    ax2.set_ylabel('time to failure', color='b')
    plt.legend(['time to failure'], loc=(0.01, 0.9))
    plt.grid(True)

plot_ad_ttf_data(train_ad_df, train_ttf_df)
del train_ad_df
del train_ttf_df

In [10]:
test_df = pd.read_csv('/kaggle/input/LANL-Earthquake-Prediction/test/seg_fb8af5.csv')
test_df.shape

# **Feature Engineering:** 
So since our data we just have two features. So we will try to add some more statistical features that might have worked for researchers in past while working with this type of data.Lets create a function to generate some statistical features based on the training data

Also as we can see that our test data has 150000 rows ,hnece we are going to split our train too in chunks/segments of 150000.

In [11]:
seg_size = 150000
segments = int(np.floor(train.shape[0] / seg_size))
SAMPLE_RATE = 4000
segments

In [16]:
def create_features(segment_id, segment, X, targets=True):
    xc = pd.Series(segment['acoustic_data'].values)
    zc = np.fft.fft(xc)
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    
    X.loc[seg_id, 'mean'] = xc.mean()
    X.loc[seg_id, 'std'] = xc.std()
    X.loc[seg_id, 'max'] = xc.max()
    X.loc[seg_id, 'min'] = xc.min()
    X.loc[seg_id, 'sum'] = xc.sum()
    X.loc[seg_id, 'mad'] = xc.mad()
    X.loc[seg_id, 'kurt'] = xc.kurtosis()
    X.loc[seg_id, 'skew'] = xc.skew()
    X.loc[seg_id, 'med'] = xc.median()
    
    X.loc[seg_id, 'q95'] = np.quantile(xc, 0.95)
    X.loc[seg_id, 'q99'] = np.quantile(xc, 0.99)
    X.loc[seg_id, 'q05'] = np.quantile(xc, 0.05)
    X.loc[seg_id, 'q01'] = np.quantile(xc, 0.01)
    
    X.loc[seg_id, 'abs_mean'] = np.abs(xc).mean()
    X.loc[seg_id, 'abs_max'] = np.abs(xc).max()
    X.loc[seg_id, 'abs_min'] = np.abs(xc).min()
    X.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X.loc[seg_id, 'Rmean'] = realFFT.mean()
    X.loc[seg_id, 'Rstd'] = realFFT.std()
    X.loc[seg_id, 'Rmax'] = realFFT.max()
    X.loc[seg_id, 'Rmin'] = realFFT.min()
    
    X.loc[seg_id, 'Imean'] = imagFFT.mean()
    X.loc[seg_id, 'Istd'] = imagFFT.std()
    X.loc[seg_id, 'Imax'] = imagFFT.max()
    X.loc[seg_id, 'Imin'] = imagFFT.min()
    
    X.loc[seg_id, 'std_first_50000'] = xc[:50000].std()
    X.loc[seg_id, 'std_last_50000'] = xc[-50000:].std()
    X.loc[seg_id, 'std_first_25000'] = xc[:25000].std()
    X.loc[seg_id, 'std_last_25000'] = xc[-25000:].std()
    X.loc[seg_id, 'std_first_10000'] = xc[:10000].std()
    X.loc[seg_id, 'std_last_10000'] = xc[-10000:].std()
    
    X.loc[seg_id, 'Rstd_first_50000'] = realFFT[:50000].std()
    X.loc[seg_id, 'Rstd_last_50000'] = realFFT[-50000:].std()
    X.loc[seg_id, 'Rstd_first_25000'] = realFFT[:25000].std()
    X.loc[seg_id, 'Rstd_last_25000'] = realFFT[-25000:].std()
    X.loc[seg_id, 'Rstd_first_10000'] = realFFT[:10000].std()
    X.loc[seg_id, 'Rstd_last_10000'] = realFFT[-10000:].std()
    
    X.loc[seg_id, 'mean_first_50000'] = xc[:50000].mean()
    X.loc[seg_id, 'mean_last_50000'] = xc[-50000:].mean()
    X.loc[seg_id, 'mean_first_25000'] = xc[:25000].mean()
    X.loc[seg_id, 'mean_last_25000'] = xc[-25000:].mean()
    X.loc[seg_id, 'mean_first_10000'] = xc[:10000].mean()
    X.loc[seg_id, 'mean_last_10000'] = xc[-10000:].mean()
    
    X.loc[seg_id, 'Rmean_first_50000'] = realFFT[:50000].mean()
    X.loc[seg_id, 'Rmean_last_50000'] = realFFT[-50000:].mean()
    X.loc[seg_id, 'Rmean_first_25000'] = realFFT[:25000].mean()
    X.loc[seg_id, 'Rmean_last_25000'] = realFFT[-25000:].mean()
    X.loc[seg_id, 'Rmean_first_10000'] = realFFT[:10000].mean()
    X.loc[seg_id, 'Rmean_last_10000'] = realFFT[-10000:].mean()
    
    X.loc[seg_id, 'max_first_50000'] = xc[:50000].max()
    X.loc[seg_id, 'max_last_50000'] = xc[-50000:].max()
    X.loc[seg_id, 'max_first_25000'] = xc[:25000].max()
    X.loc[seg_id, 'max_last_25000'] = xc[-25000:].max()
    X.loc[seg_id, 'max_first_10000'] = xc[:10000].max()
    X.loc[seg_id, 'max_last_10000'] = xc[-10000:].max()
    
    X.loc[seg_id, 'Rmax_first_50000'] = realFFT[:50000].max()
    X.loc[seg_id, 'Rmax_last_50000'] = realFFT[-50000:].max()
    X.loc[seg_id, 'Rmax_first_25000'] = realFFT[:25000].max()
    X.loc[seg_id, 'Rmax_last_25000'] = realFFT[-25000:].max()
    X.loc[seg_id, 'Rmax_first_10000'] = realFFT[:10000].max()
    X.loc[seg_id, 'Rmax_last_10000'] = realFFT[-10000:].max()
    
    X.loc[seg_id, 'min_first_50000'] = xc[:50000].min()
    X.loc[seg_id, 'min_last_50000'] = xc[-50000:].min()
    X.loc[seg_id, 'min_first_25000'] = xc[:25000].min()
    X.loc[seg_id, 'min_last_25000'] = xc[-25000:].min()
    X.loc[seg_id, 'min_first_10000'] = xc[:10000].min()
    X.loc[seg_id, 'min_last_10000'] = xc[-10000:].min()
    
    X.loc[seg_id, 'Rmin_first_50000'] = realFFT[:50000].min()
    X.loc[seg_id, 'Rmin_last_50000'] = realFFT[-50000:].min()
    X.loc[seg_id, 'Rmin_first_25000'] = realFFT[:25000].min()
    X.loc[seg_id, 'Rmin_last_25000'] = realFFT[-25000:].min()
    X.loc[seg_id, 'Rmin_first_10000'] = realFFT[:10000].min()
    X.loc[seg_id, 'Rmin_last_10000'] = realFFT[-10000:].min()
    
    #To get Rolling features we are refrencing from kernel: https://www.kaggle.com/wimwim/rolling-quantiles
    for windows in [5, 10, 50, 100, 500, 1000, 5000, 10000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values
        X.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
    if targets:
        X.loc[seg_id, 'time_to_failure'] = segment['time_to_failure'].values[-1]

In [15]:

#Calculate features and add to dataframe train_df 

train_df = pd.DataFrame(index=range(segments), dtype=np.float64)

for seg_id in tqdm_notebook(range(segments)):
    segment = train.iloc[seg_id*seg_size:seg_id*seg_size+seg_size]
    create_features(seg_id, segment, train_df)
   

In [17]:
train_df

In [18]:
X_tr=train_df.drop(['time_to_failure'],axis=1)
y_tr = train_df[['time_to_failure']].copy()
y_tr

In [19]:
plt.figure(figsize=(26, 24))
plt.title("")
cols = list(np.abs(X_tr.corrwith(y_tr['time_to_failure'])).sort_values(ascending=False).head(24).index)
for i, col in enumerate(cols):
    ax1=plt.subplot(6,4, i+1)
    plt.plot(X_tr[col], color='blue')
    plt.title(col)
    ax1.set_ylabel(col, color='b')
    ax2 = ax1.twinx()
    plt.plot(y_tr, color='g')
    ax2.set_ylabel('time_to_failure', color='g')
    plt.legend([col, 'time_to_failure'], loc=(0.875, 0.9))
    plt.grid(False)

# Since our test data is present in lots of segments hence we are going to initialize them in test_df dataframe.

In [20]:
submission = pd.read_csv('/kaggle/input/LANL-Earthquake-Prediction/sample_submission.csv', index_col='seg_id')
test_df = pd.DataFrame(columns=train_df.columns, dtype=np.float64, index=submission.index)

# Fearture Selection:
As 
There are two approaches that can be used for feature selection:
Approach 1: Correlation
Since we have large number of features, we will use the correlation matrix to select the features that have a high correlation for time_to_failure. 

In [21]:
corrMat = train_df.corr()
corrTarget = abs(corrMat['time_to_failure'])
corrTarget = corrTarget[corrTarget > 0.5]
high_corr_features = corrTarget.index.drop(['time_to_failure'])
high_corr_features

In [22]:
X_train = train_df[high_corr_features]
y_train = train_df['time_to_failure']
X_train

In [23]:
test_path = '/kaggle/input/LANL-Earthquake-Prediction/test'
for seg_id in tqdm_notebook(test_df.index):
    seg_file = seg_id + '.csv'
    seg = pd.read_csv(os.path.join(test_path, seg_file))
    create_features(seg_id, seg, test_df, targets=False)

In [24]:
final_X_test = test_df[high_corr_features]


# The function below scatters the data points by each feature and the corresponding time_to_failure 

In [25]:
def plot_feature_scatter(features, X=train_df):
    i = 0
    plt.figure()
    nlines = int(len(features)/2) 
    fig, ax = plt.subplots(nlines, 2, figsize=(20, 5*nlines ))
    for feature in features:
        i+=1
        plt.subplot(nlines,2,i)
        ax[int((i-1)/2)][(i+1) % 2].set_xlabel(feature)
        ax[int((i-1)/2)][(i+1) % 2].set_ylabel('time_to_failure')
        plt.title('{} - time_to_falure correlation)'.format(feature), color='r')
        plt.scatter(x = X[feature], y = X['time_to_failure'])
    plt.show()

In [26]:
plot_feature_scatter(high_corr_features)

The code below uses the histplot function to describe the histogram and density of the features, red for train data and blue for test data.

In [28]:
def plot_histplot_features(features, colors=['red', 'blue'], df1=X_train):
    i = 0
    plt.figure()
    nlines = int(len(features)/2)
    fig, ax = plt.subplots(nlines,2,figsize=(20,5*nlines))
    for feature in features:
        i += 1
        plt.subplot(nlines,2,i)
        sns.histplot(df1[feature],color=colors[0], kde=True, bins=40, label='train')
        
    plt.show()

In [29]:
plot_histplot_features(high_corr_features)

Now we will scale the features to better fit the normal distribution and will try to visualize our data again. We will scale with both train and test data. Yellow corresponds to train data and Red corresponds to test data. 

In [30]:
scaler = StandardScaler()
scaler.fit(pd.concat([X_train, final_X_test]))
scaled_X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
scaled_X_test = pd.DataFrame(scaler.transform(final_X_test), columns=final_X_test.columns, index=final_X_test.index)

In [31]:
plot_histplot_features(high_corr_features, colors='yellow', df1=scaled_X_train)

# **After scaling our features we will plot the highly correlated features with time of failure.**

In [33]:
def plot_feature_ttf(features):
    i = 0
    plt.figure()
    nlines = int(len(features)/2)
    fig, ax = plt.subplots(nlines,2,figsize=(32,8*nlines))
    for feature in features:
        i += 1
        plt.subplot(nlines,2,i)
        plt.title('({}) and time to failure'.format(feature))
        plt.plot(X_train[feature], color='r')
        ax[int((i-1)/2)][(i+1) % 2].set_xlabel('training samples')
        ax[int((i-1)/2)][(i+1) % 2].set_ylabel('acoustic data ({})'.format(feature), color='r')
        plt.legend(['acoustic data ({})'.format(feature)], loc=(0.01, 0.95))
        ax2 = ax[int((i-1)/2)][(i+1) % 2].twinx()
        plt.plot(y_train, color='b')
        ax2.set_ylabel('time to failure', color='b')
        plt.legend(['time to failure'], loc=(0.01, 0.9))
        plt.grid(True)
    plt.show()

In [34]:
plot_feature_ttf(high_corr_features)

# Observation: 
We can clearly see that during the time of failure there is spike in the values of the statistically generated features.

In [35]:
X_train, X_valid, y_train, y_valid = train_test_split(scaled_X_train, y_tr, test_size=0.33, random_state=42)

# SVM+RBF Kernel model

We are using gridsearch CV for Hyperparameter tuning and find the best C and epsilon value.

In [ ]:
parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[0.01, 0.1, 1, 10, 100],'epsilon':[0.1,0.2,0.3,0.4,0.5,0.6]}
svr = SVR(gamma = 'scale')
clf = GridSearchCV(svr, parameters)
clf.fit(X_train,y_train)
clf.best_params_

In [ ]:
model = SVR(kernel='rbf', C=1, epsilon=0.6, gamma='scale')
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_valid)
mae = mean_absolute_error(y_pred, y_valid)
print('mae = {}'.format(mae))

# CatBoost algorithm

In [ ]:
train_pool = Pool(X_train, y_train)
prediction = np.zeros(len(X_valid))
m = CatBoostRegressor(iterations=10000, loss_function='MAE', boosting_type='Ordered')
m.fit(X_train, y_train, silent=True)
m.best_score_

In [1]:
model_file = "model.pickle"
with open(model_file,'wb') as f:
    pickle.dump(m, f)

In [2]:
joblib.dump(m,'model.pkl')

# XgBoost Algorithm

In [ ]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train,y_train.values)
pred = xgb_model.predict(X_valid)#prediction without hyperparameter tuning


In [ ]:
# hyperparameter tuning with XGBoost 
# creating a KFold object with 3 splits

folds = KFold(n_splits= 3, shuffle= True, random_state= 101)

# specify range of hyperparameters
param_grid = {'learning_rate': [0.01, 0.1, 0.2, 0.3], 
             'subsample': [0.3, 0.6, 0.9, 1],
              'n_estimators' : [5, 10, 15, 20],
              'max_depth' :[2,4,6,8]          
             }          


# specify model
xgb_model = xgb.XGBRegressor()

# set up GridSearchCV()
model_cv = GridSearchCV(estimator = xgb_model, 
                        param_grid = param_grid, 
                        scoring='neg_mean_absolute_error', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True, 
                        n_jobs= -1)      

In [ ]:
model_cv.fit(X_train,y_train.values)   #train the model

In [ ]:
print(' neg_mean_absolute_error:',model_cv.best_score_,'using',model_cv.best_params_)

In [ ]:
#define the model with the best parameter and training oif the model to make the predictions
xgb_model = xgb.XGBRegressor(learning_rate= 0.2, max_depth= 2, n_estimators= 15, subsample= 0.9)

xgb_model.fit(X_train,y_train.values)

pred = xgb_model.predict(X_valid)
#pred

In [ ]:
mytable=PrettyTable(["Model","Best hyperparameter","MAE"])
mytable.add_row(["SVR+RBF","C=1,epsilon=0.6","2.091950549780848"])
mytable.add_row(["CatBoost Regressor","iteration=10000","1.0722845986656637"])
mytable.add_row(["XBoost","Kfold=5","2.0792648604931623"])

Since we are getting best mean absolute error for catboost, Hence we will be using the same for predictions.

In [ ]:
prediction=m.predict(final_X_test)
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
submission['time_to_failure'] = prediction
submission.to_csv('submission_cat.csv', index=False)
submission.head()